In [ ]:
import pandas as pd
import numpy as np

# Create df with header

In [ ]:
df = pd.read_csv('results.csv', encoding = "ISO-8859-1", names = ['info', 'dL (mm)', 'F (kN)',
                    'specimen', 'strain (%)', 'stress (MPa)'])

# Remove NaN (change to empty)

In [ ]:
df = df.fillna('')

# Store thickness values for each specimen

In [ ]:
thicknesses = [float(line.replace('    Spessore:         ', '').replace(' mm', ''))
                for line in df['info'] if 'Spessore' in line]

# Drop all rows that do not contain test data

In [ ]:
df = df[df['dL (mm)'] != '']

# Populate specimen col

In [ ]:
specimens = []
thick_col = []
counter = 0

for line in df['info']:
    if line == '1':
        counter += 1
    specimens.append(counter)
    thick_col.append(thicknesses[counter - 1])

df['specimen'] = specimens

# Populate strain col

In [ ]:
length_0 = 30 # initial length in mm

df['strain (%)'] = df['dL (mm)'] / length_0 * 100

# Populate stress col

In [ ]:
width = 10 # width in mm
    
df['stress (MPa)'] = df['F (kN)'] * 1000 / width / thick_col

df = df.reset_index(drop=True)

# Tensile Modulus (ISO 527-1,2)

In [ ]:
# for tensile modulus consider ISO 527, i.e., calculate slope for strain range [0.05% 0.25%]

df_mod_calc = df[(df['strain (%)'] <= 0.25) & (df['strain (%)'] >= 0.05)]

tensile_mod = []

for i in range(len(thicknesses)):
    xy = df_mod_calc[df_mod_calc['specimen'] == i + 1]
    x = xy['strain (%)'].tolist()
    y = xy['stress (MPa)'].tolist()
    slope, intercept = np.polyfit(x, y, 1)
    print(slope)
    tensile_mod.append(slope*100)
    print(tensile_mod)